# Retry Logic Test Suite
This notebook tests the retry decorators to ensure they work correctly for Azure OpenAI and Azure Search scenarios.

## 1. Setup - Import Retry Logic

In [ ]:
import time
import random
from datetime import datetime
from functools import wraps
import pandas as pd
import matplotlib.pyplot as plt

# Import the retry logic from your optimization file
# If this doesn't work, copy the retry functions from optimization_solutions.py
try:
    from optimization_solutions import (
        retry_with_exponential_backoff,
        retry_azure_openai,
        retry_azure_search
    )
    print("Imported retry logic from optimization_solutions.py")
except ImportError:
    print("Could not import from optimization_solutions.py")
    print("Copy the retry logic functions here if needed")

## 2. Test Metrics Tracker

In [ ]:
class TestMetrics:
    """Track test execution metrics"""
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.attempts = 0
        self.timestamps = []
        self.errors = []
        self.delays = []
    
    def record_attempt(self, error=None):
        self.attempts += 1
        current_time = datetime.now()
        self.timestamps.append(current_time)
        
        if len(self.timestamps) > 1:
            delay = (current_time - self.timestamps[-2]).total_seconds()
            self.delays.append(delay)
        
        if error:
            self.errors.append(str(error))
    
    def get_summary(self):
        return {
            'attempts': self.attempts,
            'total_time': (self.timestamps[-1] - self.timestamps[0]).total_seconds() if len(self.timestamps) > 1 else 0,
            'retry_delays': self.delays,
            'errors': self.errors
        }

# Global metrics instance
metrics = TestMetrics()

## 3. Test Scenario 1: Rate Limit (Should Retry)

In [ ]:
# Test function that simulates rate limit errors
@retry_azure_openai(max_retries=3)
def test_rate_limit(fail_times=2):
    """Simulates Azure OpenAI rate limit errors"""
    metrics.record_attempt()
    
    if metrics.attempts <= fail_times:
        error = Exception("Error 429: Rate limit exceeded")
        metrics.record_attempt(error)
        raise error
    
    return f"Success after {metrics.attempts} attempts"

# Run the test
print("TEST: Rate Limit Scenario")
print("Expected: Should retry with exponential backoff and succeed")
print("-" * 50)

metrics.reset()
start_time = time.time()

try:
    result = test_rate_limit(fail_times=2)
    print(f"✓ SUCCESS: {result}")
    print(f"Total time: {time.time() - start_time:.2f} seconds")
    print(f"Retry delays: {metrics.delays}")
except Exception as e:
    print(f"✗ FAILED: {str(e)}")

## 4. Test Scenario 2: Timeout (Should Retry)

In [ ]:
# Test function that simulates timeout errors
@retry_azure_search(max_retries=3)
def test_timeout(fail_times=2):
    """Simulates Azure Search timeout errors"""
    metrics.record_attempt()
    
    if metrics.attempts <= fail_times:
        error = Exception("Request timeout after 30 seconds")
        metrics.record_attempt(error)
        raise error
    
    return f"Success after {metrics.attempts} attempts"

# Run the test
print("TEST: Timeout Scenario")
print("Expected: Should retry with shorter delays (Azure Search)")
print("-" * 50)

metrics.reset()
start_time = time.time()

try:
    result = test_timeout(fail_times=2)
    print(f"✓ SUCCESS: {result}")
    print(f"Total time: {time.time() - start_time:.2f} seconds")
    print(f"Retry delays: {metrics.delays}")
except Exception as e:
    print(f"✗ FAILED: {str(e)}")

## 5. Test Scenario 3: Non-Retriable Errors

In [ ]:
# Test unauthorized error (should NOT retry)
@retry_azure_openai(max_retries=3)
def test_unauthorized():
    """Simulates unauthorized error"""
    metrics.record_attempt()
    error = Exception("Error 401: Unauthorized - invalid API key")
    raise error

# Test quota exceeded (should NOT retry)
@retry_azure_openai(max_retries=3)
def test_quota_exceeded():
    """Simulates quota exceeded error"""
    metrics.record_attempt()
    error = Exception("Error 403: Quota exceeded for this month")
    raise error

# Run non-retriable tests
for test_name, test_func in [("Unauthorized", test_unauthorized), ("Quota Exceeded", test_quota_exceeded)]:
    print(f"\nTEST: {test_name}")
    print("Expected: Should fail immediately without retry")
    print("-" * 50)
    
    metrics.reset()
    start_time = time.time()
    
    try:
        result = test_func()
        print(f"✗ UNEXPECTED SUCCESS")
    except Exception as e:
        elapsed = time.time() - start_time
        if metrics.attempts == 1 and elapsed < 1:
            print(f"✓ CORRECTLY FAILED: {str(e)}")
            print(f"Attempts: {metrics.attempts} (no retries as expected)")
        else:
            print(f"✗ INCORRECTLY RETRIED: {metrics.attempts} attempts")

## 6. Test Scenario 4: Exhaust All Retries

In [ ]:
# Test function that always fails
@retry_azure_openai(max_retries=2)
def test_always_fails():
    """Function that always fails to test retry exhaustion"""
    metrics.record_attempt()
    error = Exception("Service unavailable")
    raise error

# Run the test
print("TEST: Exhaust All Retries")
print("Expected: Should retry max_retries times then fail")
print("-" * 50)

metrics.reset()
start_time = time.time()

try:
    result = test_always_fails()
    print(f"✗ UNEXPECTED SUCCESS")
except Exception as e:
    print(f"✓ CORRECTLY FAILED: {str(e)}")
    print(f"Total attempts: {metrics.attempts}")
    print(f"Total time: {time.time() - start_time:.2f} seconds")
    print(f"Retry delays: {metrics.delays}")

## 7. Visualize Retry Delays

In [ ]:
# Test with different retry counts to see exponential backoff
test_results = []

for fail_times in range(1, 4):
    @retry_azure_openai(max_retries=3)
    def test_backoff():
        metrics.record_attempt()
        if metrics.attempts <= fail_times:
            raise Exception("Rate limit exceeded")
        return "Success"
    
    metrics.reset()
    try:
        test_backoff()
        test_results.append({
            'fail_times': fail_times,
            'total_attempts': metrics.attempts,
            'retry_delays': metrics.delays,
            'total_time': sum(metrics.delays) if metrics.delays else 0
        })
    except:
        pass

# Visualize the exponential backoff
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot retry delays
for result in test_results:
    if result['retry_delays']:
        attempts = list(range(1, len(result['retry_delays']) + 1))
        ax1.plot(attempts, result['retry_delays'], marker='o', label=f"{result['fail_times']} failures")

ax1.set_xlabel('Retry Attempt')
ax1.set_ylabel('Delay (seconds)')
ax1.set_title('Exponential Backoff Pattern')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot total time
fail_counts = [r['fail_times'] for r in test_results]
total_times = [r['total_time'] for r in test_results]
ax2.bar(fail_counts, total_times, color='steelblue')
ax2.set_xlabel('Number of Failures Before Success')
ax2.set_ylabel('Total Retry Time (seconds)')
ax2.set_title('Total Time Spent in Retries')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nRetry Delay Analysis:")
for result in test_results:
    print(f"  {result['fail_times']} failures: {[f'{d:.2f}s' for d in result['retry_delays']]}")

## 8. Performance Comparison: With vs Without Retry

In [ ]:
# Simulate a real-world scenario with intermittent failures
import numpy as np

def simulate_api_call(failure_rate=0.3):
    """Simulates an API call with given failure rate"""
    if random.random() < failure_rate:
        raise Exception("Rate limit exceeded")
    return "Success"

# Test without retry
successes_without_retry = 0
failures_without_retry = 0
total_calls = 100

for _ in range(total_calls):
    try:
        simulate_api_call(failure_rate=0.3)
        successes_without_retry += 1
    except:
        failures_without_retry += 1

# Test with retry
@retry_azure_openai(max_retries=3)
def simulate_api_call_with_retry(failure_rate=0.3):
    if random.random() < failure_rate:
        raise Exception("Rate limit exceeded")
    return "Success"

successes_with_retry = 0
failures_with_retry = 0

for _ in range(total_calls):
    try:
        simulate_api_call_with_retry(failure_rate=0.3)
        successes_with_retry += 1
    except:
        failures_with_retry += 1

# Display results
comparison_df = pd.DataFrame({
    'Metric': ['Success Rate', 'Failure Rate', 'Successful Calls', 'Failed Calls'],
    'Without Retry': [
        f"{(successes_without_retry/total_calls)*100:.1f}%",
        f"{(failures_without_retry/total_calls)*100:.1f}%",
        successes_without_retry,
        failures_without_retry
    ],
    'With Retry': [
        f"{(successes_with_retry/total_calls)*100:.1f}%",
        f"{(failures_with_retry/total_calls)*100:.1f}%",
        successes_with_retry,
        failures_with_retry
    ]
})

print("\nPerformance Comparison (100 API calls with 30% failure rate):")
display(comparison_df)

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
x = np.arange(2)
width = 0.35

without_retry = [successes_without_retry, failures_without_retry]
with_retry = [successes_with_retry, failures_with_retry]

bars1 = ax.bar(x - width/2, without_retry, width, label='Without Retry', color=['green', 'red'])
bars2 = ax.bar(x + width/2, with_retry, width, label='With Retry', color=['darkgreen', 'darkred'])

ax.set_ylabel('Number of Calls')
ax.set_title('Impact of Retry Logic on API Call Success Rate')
ax.set_xticks(x)
ax.set_xticklabels(['Successes', 'Failures'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

improvement = ((successes_with_retry - successes_without_retry) / successes_without_retry) * 100
print(f"\nImprovement with retry logic: {improvement:.1f}% more successful calls")

## 9. Test Summary

In [ ]:
# Run all tests and summarize
test_suite = [
    ("Rate Limit Recovery", test_rate_limit, {'fail_times': 2}, True),
    ("Timeout Recovery", test_timeout, {'fail_times': 1}, True),
    ("Unauthorized No Retry", test_unauthorized, {}, False),
    ("Quota Exceeded No Retry", test_quota_exceeded, {}, False),
    ("Exhaust Retries", test_always_fails, {}, False)
]

results = []

print("COMPLETE TEST SUITE RESULTS")
print("=" * 60)

for test_name, test_func, kwargs, should_succeed in test_suite:
    metrics.reset()
    
    try:
        result = test_func(**kwargs)
        actual_result = "Success"
        status = "PASS" if should_succeed else "FAIL"
    except:
        actual_result = "Failed"
        status = "PASS" if not should_succeed else "FAIL"
    
    results.append({
        'Test': test_name,
        'Expected': "Success" if should_succeed else "Failure",
        'Actual': actual_result,
        'Status': status,
        'Attempts': metrics.attempts
    })

results_df = pd.DataFrame(results)
display(results_df)

passed = len(results_df[results_df['Status'] == 'PASS'])
total = len(results_df)

print(f"\nTest Results: {passed}/{total} tests passed")

if passed == total:
    print("✅ All tests passed! The retry logic is working correctly.")
else:
    print(f"⚠️ {total - passed} tests failed. Review the retry implementation.")